<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/usecases/10k_sub_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10K Analysis
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [52]:
!pip install llama-index
!pip install llama-index.response

ERROR: Could not find a version that satisfies the requirement llama-index.response (from versions: none)
ERROR: No matching distribution found for llama-index.response


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [53]:
"""from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI """
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.core.response.notebook_utils import display_response
from IPython.display import Markdown, display

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

## Configure LLM service

In [59]:
import os
from google.colab import userdata
userdata.get('chatgpt')

os.environ["OPENAI_API_KEY"] = userdata.get('chatgpt')

NameError: name 'llm' is not defined

In [61]:
from llama_index.core import Settings
llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")
Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")



ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

## Download Data

In [56]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-12-17 04:16:28--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.09s   

2024-12-17 04:16:29 (18.9 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-12-17 04:16:29--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

## Load data

In [65]:
from google.colab import drive
drive.mount('/content/drive')

lyft_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/A9/Lyft10Q2024Q3.pdf"]
).load_data()
uber_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/A9/Uber10Q2024Q3.pdf"]
).load_data()

Mounted at /content/drive


## Build indices

In [66]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)

In [67]:
uber_index = VectorStoreIndex.from_documents(uber_docs)

## Build query engines

In [68]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [69]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [73]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10q",
            description=(
                "Provides information about Lyft financials for Q3 2024"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10q",
            description=(
                "Provides information about Uber financials for Q3 2024"
            ),
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

## Run queries

In [74]:
response = s_engine.query(
    "Compare and contrast the customer segments and geographies that grew the"
    " fastest"
)

Generated 4 sub questions.
[lyft_10q] Q: What were the customer segments that grew the fastest for Lyft in Q3 2024?
[lyft_10q] Q: What were the geographies that grew the fastest for Lyft in Q3 2024?
[uber_10q] Q: What were the customer segments that grew the fastest for Uber in Q3 2024?
[uber_10q] Q: What were the geographies that grew the fastest for Uber in Q3 2024?
[lyft_10q] A: In Q3 2024, the customer segments that grew the fastest for Lyft were women and nonbinary drivers and riders, as indicated by the launch and extension of the Women+ Connect feature nationwide.
[uber_10q] A: Chicago, Los Angeles, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom were the geographies that grew the fastest for Uber in Q3 2024.
[uber_10q] A: The customer segments that grew the fastest for Uber in Q3 2024 were Mobility and Delivery.
[lyft_10q] A: The geographies that grew the fastest for Lyft in Q3 2024 were not explicitly mentioned in the provided context 

In [72]:
print(response)

The customer segments that experienced the fastest growth for Lyft in 2021 were Active Riders and Rides, while for Uber, it was the Delivery segment and the Mobility segment. In terms of geographies, Lyft expanded into new areas with the help of existing rider referrals, whereas Uber saw significant growth in cities like Chicago, Los Angeles, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom.


In [80]:
response = s_engine.query(
    "Looking at Uber's balance sheet, what is Uber's total liabilities?"
)

Generated 2 sub questions.
[lyft_10q] Q: What is Lyft's total liabilities for Q3 2024?
[uber_10q] Q: What is Uber's total liabilities for Q3 2024?
[lyft_10q] A: Lyft's total liabilities for Q3 2024 are $4,607,376.
[uber_10q] A: Uber's total liabilities for Q3 2024 were $30,578 million.


In [81]:
import pandas as pd
from tabulate import tabulate

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

metrics = [
    "revenue",
    "net income",
    "operating income",
    "total assets",
    "total liabilities from balance sheet",
    "total debt",
    "total stockholders' equity"
    "gross profit margin",
    "price/earnings ratio",
    "return on equity",
    "working capital",
    "operating cash flow"
]

report_data = []
for metric in metrics:
    response_lyft = s_engine.query(f"What is the {metric} for Lyft in Q3 2024? Answer only in integer or NA.")
    response_uber = s_engine.query(f"What is the {metric} for Uber in Q3 2024? Answer only in integer or NA.")
    report_data.append([
        metric,
        response_lyft.response,
        response_uber.response
        ])

Generated 1 sub questions.
[lyft_10q] Q: What is the revenue for Lyft in Q3 2024?
[lyft_10q] A: The revenue for Lyft in Q3 2024 was $1,522,692,000.
Generated 1 sub questions.
[uber_10q] Q: What is the revenue for Uber in Q3 2024?
[uber_10q] A: $11,188 million
Generated 1 sub questions.
[lyft_10q] Q: What is the net income for Lyft in Q3 2024?
[lyft_10q] A: Lyft achieved a net income for the first time in its operating history in the second quarter of 2024.
Generated 1 sub questions.
[uber_10q] Q: What is the net income for Uber in Q3 2024?
[uber_10q] A: $2.6 billion.
Generated 1 sub questions.
[lyft_10q] Q: What is the operating income for Lyft in Q3 2024?
[lyft_10q] A: The operating income for Lyft in Q3 2024 is a loss of $56,687,000.
Generated 1 sub questions.
[uber_10q] Q: What is the operating income for Uber in Q3 2024?
[uber_10q] A: The operating income for Uber in Q3 2024 is $1.061 billion.
Generated 1 sub questions.
[lyft_10q] Q: What is the total assets for Lyft in Q3 2024?
[l

In [82]:
df = pd.DataFrame(report_data, columns=["Metric", "Lyft", "Uber"])
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------------------------------+----------------+-----------------------------------------------------------------+
|    | Metric                               | Lyft           | Uber                                                            |
|----+--------------------------------------+----------------+-----------------------------------------------------------------|
|  0 | revenue                              | $1,522,692,000 | $11,188 million                                                 |
|  1 | net income                           | NA             | $2.6 billion                                                    |
|  2 | operating income                     | -56687000      | $1.061 billion                                                  |
|  3 | total assets                         | $5,263,155     | The total assets for Uber in Q3 2024 amount to $47,117,000,000. |
|  4 | total liabilities from balance sheet | 4607376        | $30,578 million                   

In [63]:
print(response)

The operating cash flow for Lyft in 2021 was a net cash used of $101,721,000, and for Uber in 2021, it was $(445) million.


Items to report:
- revenue
- net income
- operating income
- total assets
- total liabilities
- debt to equity ratio = total debt / shareholder's equity
- gross profit margin = (net income/revenue) x 100%
- price/earnings ratio = share price / earnings per share
- return on equity
- working capital
- operting cash flow
